## 2. Fairness

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from xgboost import XGBClassifier
import shap

from aif360.metrics import BinaryLabelDatasetMetric, ClassificationMetric
from aif360.datasets import AdultDataset, BinaryLabelDataset
from aif360.algorithms.preprocessing import Reweighing
from aif360.algorithms.preprocessing.optim_preproc_helpers.data_preproc_functions import load_preproc_data_adult

from IPython.display import Markdown, display

%matplotlib inline


c:\Users\theod\anaconda3\envs\omnixai\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\theod\anaconda3\envs\omnixai\Lib\site-packages\inFairness\utils\ndcg.py:37: FutureWarning: We've integrated functorch into PyTorch. As the final step of the integration, `functorch.vmap` is deprecated as of PyTorch 2.0 and will be deleted in a future version of PyTorch >= 2.3. Please use `torch.vmap` instead; see the PyTorch 2.0 release notes and/or the `torch.func` migration guide for more details https://pytorch.org/docs/main/func.migrating.html
  vect_normalized_discounted_cumulative_gain = vmap(
c:\Users\theod\anaconda3\envs\omnixai\Lib\site-packages\inFairness\utils\ndcg.py:48: FutureWarning: We've integrated functorch into PyTorch. As the final step of the integration, `functorch.vmap` is deprecated as of PyTorch 2

Load Dataset

In [3]:
np.random.seed(1)   #random seed for calibrated equal odds prediction
columns = [
    "age", "workclass", "fnlwgt", "education", "education-num",
    "marital-status", "occupation", "relationship", "race", "sex",
    "capital-gain", "capital-loss", "hours-per-week", "native-country", "income"
]
# We want to difine a custom preprocesing function (custom_preprocessing(df)) from the standat dataset class 
# that will be used to transform the dataset

def custom_preprocessing(df):
    median_age = df['age'].median()
    df['age_binary'] = df['age'].apply(lambda x: 0 if x <= median_age else 1)
    df = df.drop('age', axis=1)
    df['race'] = df['race'].apply(lambda x: 1 if x =="White"  else 0)
    df['sex'] =df['sex'].apply(lambda x: 1 if x =="Male"  else 0)
    return df
# So what we did is to add a new column 'age_binary' to the dataset and drop the 'age' column, in order to 
# binarise the age column.
# Load the dataset with the library aif360
dataset= AdultDataset(custom_preprocessing=custom_preprocessing,
                          protected_attribute_names=['age_binary', 'sex'], # race will remain because in the original library is defined with this protecte attribute
                          privileged_classes=[np.array([1.0]),np.array([1.0]) ]) # We supposed that the privileged class is the old white male. It's also defined like this in the original library

# Get the dataset and split into train and test
dataset_orig_train, dataset_orig_vt = dataset.split([0.7], shuffle=True)
dataset_orig_valid, dataset_orig_test = dataset_orig_vt.split([0.5], shuffle=True)


#### Clean up training data

In [4]:
# print out some labels, names, etc.
display(Markdown("#### Training Dataset shape"))
print(dataset_orig_train.features.shape)
display(Markdown("#### Favorable and unfavorable labels"))
print(dataset_orig_train.favorable_label, dataset_orig_train.unfavorable_label)
display(Markdown("#### Protected attribute names"))
print(dataset_orig_train.protected_attribute_names)

display(Markdown("#### Privileged and unprivileged protected attribute values"))
print(dataset_orig_train.privileged_protected_attributes, 
      dataset_orig_train.unprivileged_protected_attributes)

display(Markdown("#### Dataset feature names"))
print(dataset_orig_train.feature_names)

#### Training Dataset shape

(31655, 98)


#### Favorable and unfavorable labels

1.0 0.0


#### Protected attribute names

['age_binary', 'sex']


#### Privileged and unprivileged protected attribute values

[array([1.]), array([1.])] [array([0.]), array([0.])]


#### Dataset feature names

['education-num', 'race', 'sex', 'capital-gain', 'capital-loss', 'hours-per-week', 'age_binary', 'workclass=Federal-gov', 'workclass=Local-gov', 'workclass=Private', 'workclass=Self-emp-inc', 'workclass=Self-emp-not-inc', 'workclass=State-gov', 'workclass=Without-pay', 'education=10th', 'education=11th', 'education=12th', 'education=1st-4th', 'education=5th-6th', 'education=7th-8th', 'education=9th', 'education=Assoc-acdm', 'education=Assoc-voc', 'education=Bachelors', 'education=Doctorate', 'education=HS-grad', 'education=Masters', 'education=Preschool', 'education=Prof-school', 'education=Some-college', 'marital-status=Divorced', 'marital-status=Married-AF-spouse', 'marital-status=Married-civ-spouse', 'marital-status=Married-spouse-absent', 'marital-status=Never-married', 'marital-status=Separated', 'marital-status=Widowed', 'occupation=Adm-clerical', 'occupation=Armed-Forces', 'occupation=Craft-repair', 'occupation=Exec-managerial', 'occupation=Farming-fishing', 'occupation=Handlers

Step 3 Compute fairness metric on original training dataset
The fairness metric is Statistical Parity Difference whitch measures the disparity in positive outcomes between unprivileged and privileged groups. It compares the probability of receiving a positive outcome for members of the unprivileged group against that for members of the privileged group.

In [5]:
# Define the privileged and unprivileged groups in order to compute the disparate impact
privileged_groups = [{'age_binary': 1, 'sex': 1}]  # Old males
unprivileged_groups = [{'age_binary': 0, 'sex': 0}]  # Young females


As it can be seen from the privious cell, we conclude that there is a bias in this dataset because the statistical paity metric is not equal to zero. More specifficaly, in the unprivileged group we have 2% of peopele are suffering of unfairness.  


 Step 4 Mitigate bias by transforming the original dataset via technique to ensure the classifier is fair. Here we want to use the Pre-Processing method Reweighting for fairness. This method will simply assigns weights to samples to balance the representation of protected groups in the training process.

In [6]:
# Compute the fairness metric statistical parity measure, which is the difference in the mean prediction between the unprivileged and privileged groups.
# A negative value indicates less favorable outcomes for the unprivileged groups. in order to see if the dataset is biased
metric_orig_train = BinaryLabelDatasetMetric(dataset_orig_train, 
                                             unprivileged_groups=unprivileged_groups,
                                             privileged_groups=privileged_groups)
display(Markdown("#### Original training dataset"))
print("Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_orig_train.mean_difference())

#### Original training dataset

Difference in mean outcomes between unprivileged and privileged groups = -0.363363


In [7]:
print('The 50 first instance weights originally:')
dataset.instance_weights[:50]

The 50 first instance weights originally:


array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

We can clearly see from the cell above that the reweight method of transforming the dataset into a fair one worked.


Step 5 Compute fairness metric on transformed dataset


In [8]:
RW = Reweighing(unprivileged_groups=unprivileged_groups,
               privileged_groups=privileged_groups)
RW.fit(dataset_orig_train)
dataset_transf_train = RW.transform(dataset_orig_train)

In [9]:
print('The 50 first instance weights after reweighing:')
dataset_transf_train.instance_weights[:50]

The 50 first instance weights after reweighing:


array([1.        , 1.33826683, 1.        , 1.        , 1.        ,
       1.        , 0.81264928, 1.        , 1.        , 1.33826683,
       1.33826683, 1.        , 1.        , 1.33826683, 1.        ,
       1.33826683, 1.        , 1.        , 1.        , 1.33826683,
       0.56633898, 1.33826683, 1.        , 0.81264928, 1.        ,
       1.        , 0.81264928, 1.        , 1.33826683, 1.33826683,
       0.81264928, 0.81264928, 0.56633898, 1.33826683, 0.81264928,
       1.33826683, 0.56633898, 0.81264928, 1.33826683, 0.56633898,
       3.31574203, 0.81264928, 1.        , 3.31574203, 1.        ,
       1.        , 1.        , 1.        , 0.56633898, 1.        ])

In [10]:
metric_transf_train = BinaryLabelDatasetMetric(dataset_transf_train, 
                                               unprivileged_groups=unprivileged_groups,
                                               privileged_groups=privileged_groups)
display(Markdown("#### Transformed training dataset"))
print("Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_transf_train.mean_difference())


#### Transformed training dataset

Difference in mean outcomes between unprivileged and privileged groups = -0.000000


### Train classifier on original data


In [11]:
# Extract data for the original training set
X_train = dataset_orig_train.features
y_train = dataset_orig_train.labels.ravel()

X_valid = dataset_orig_valid.features
y_valid = dataset_orig_valid.labels.ravel()

X_test = dataset_orig_test.features
y_test = dataset_orig_test.labels.ravel()

# Extract data for the reweighted (fair) training set
X_train_transf = dataset_transf_train.features
y_train_transf = dataset_transf_train.labels.ravel()
w_train_transf = dataset_transf_train.instance_weights

# Prepare a scaler to normalize features (helpful for logistic regression)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_valid_scaled = scaler.transform(X_valid)
X_test_scaled = scaler.transform(X_test)

X_train_transf_scaled = scaler.fit_transform(X_train_transf)
X_valid_transf_scaled = scaler.transform(X_valid)  # validation set stays the same
X_test_transf_scaled = scaler.transform(X_test)

In [12]:
# ===========================
# Model Without Fairness Mitigation
# ===========================
clf_orig = LogisticRegression(solver='lbfgs', max_iter=200)
clf_orig.fit(X_train_scaled, y_train)

y_pred_test_orig = clf_orig.predict(X_test_scaled)

# Evaluate performance
print("===== Original Classifier (No Fairness Mitigation) =====")
print("Accuracy on test set:", accuracy_score(y_test, y_pred_test_orig))
print(classification_report(y_test, y_pred_test_orig))

# Compute fairness metrics on test set
test_bld_orig = dataset_orig_test.copy(deepcopy=True)
test_bld_orig.labels = y_pred_test_orig.reshape(-1,1)

metric_test_orig = ClassificationMetric(dataset_orig_test,
                                        test_bld_orig,
                                        unprivileged_groups=unprivileged_groups,
                                        privileged_groups=privileged_groups)
print("Statistical parity difference (original):", metric_test_orig.statistical_parity_difference())
print("Equal opportunity difference (original):", metric_test_orig.equal_opportunity_difference())

===== Original Classifier (No Fairness Mitigation) =====
Accuracy on test set: 0.8536261792452831
              precision    recall  f1-score   support

         0.0       0.88      0.93      0.90      5079
         1.0       0.75      0.63      0.68      1705

    accuracy                           0.85      6784
   macro avg       0.81      0.78      0.79      6784
weighted avg       0.85      0.85      0.85      6784

Statistical parity difference (original): -0.3649024190849286
Equal opportunity difference (original): -0.1068310220852593


In [13]:
# ===========================
# Model With Fairness Mitigation (Reweighted)
# ===========================
clf_transf = LogisticRegression(solver='lbfgs', max_iter=200)
# Important: use the instance weights when training on the reweighted dataset
clf_transf.fit(X_train_transf_scaled, y_train_transf, sample_weight=w_train_transf)

y_pred_test_transf = clf_transf.predict(X_test_transf_scaled)

print("\n===== Fairness Mitigated Classifier (Reweighted) =====")
print("Accuracy on test set:", accuracy_score(y_test, y_pred_test_transf))
print(classification_report(y_test, y_pred_test_transf))

# Compute fairness metrics on test set for the fairness mitigated classifier
test_bld_transf = dataset_orig_test.copy(deepcopy=True)
test_bld_transf.labels = y_pred_test_transf.reshape(-1,1)

metric_test_transf = ClassificationMetric(dataset_orig_test,
                                          test_bld_transf,
                                          unprivileged_groups=unprivileged_groups,
                                          privileged_groups=privileged_groups)
print("Statistical parity difference (reweighted):", metric_test_transf.statistical_parity_difference())
print("Equal opportunity difference (reweighted):", metric_test_transf.equal_opportunity_difference())


===== Fairness Mitigated Classifier (Reweighted) =====
Accuracy on test set: 0.8325471698113207
              precision    recall  f1-score   support

         0.0       0.85      0.95      0.89      5079
         1.0       0.76      0.49      0.60      1705

    accuracy                           0.83      6784
   macro avg       0.80      0.72      0.75      6784
weighted avg       0.82      0.83      0.82      6784

Statistical parity difference (reweighted): -0.14408719185335156
Equal opportunity difference (reweighted): 0.3036723163841808


In [14]:
# ===========================
# Comparison & Conclusions
# ===========================
print("\nComparison of Results:")
print("Original vs Reweighted:")
print(" - Test Accuracy: {:.4f} vs {:.4f}".format(accuracy_score(y_test, y_pred_test_orig), accuracy_score(y_test, y_pred_test_transf)))
print(" - Statistical Parity Difference: {:.4f} vs {:.4f}".format(metric_test_orig.statistical_parity_difference(),
                                                                  metric_test_transf.statistical_parity_difference()))
print(" - Equal Opportunity Difference: {:.4f} vs {:.4f}".format(metric_test_orig.equal_opportunity_difference(),
                                                                 metric_test_transf.equal_opportunity_difference()))




Comparison of Results:
Original vs Reweighted:
 - Test Accuracy: 0.8536 vs 0.8325
 - Statistical Parity Difference: -0.3649 vs -0.1441
 - Equal Opportunity Difference: -0.1068 vs 0.3037


In [15]:
# Convert the datasets to pandas DataFrame
df = dataset_transf_train.convert_to_dataframe()[0]

# Divide into features (X) and labels (y)
X = df.drop('income-per-year', axis=1)
y = df['income-per-year']

# Divide into training and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Train an XGBoost classifier
model = XGBClassifier(eval_metric='logloss', random_state=42)
model.fit(X_train, y_train)

# Predictions
y_pred = model.predict(X_test)

# Evaluate performance
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))

Accuracy: 0.8645888175213226
Classification Report:
               precision    recall  f1-score   support

         0.0       0.89      0.94      0.91      7130
         1.0       0.77      0.65      0.70      2367

    accuracy                           0.86      9497
   macro avg       0.83      0.79      0.81      9497
weighted avg       0.86      0.86      0.86      9497

